In [1]:
from manim import *
from objects import *

In [2]:
%%manim -ql Test
from processing import *
from animations import *

dots_color=[RED, ManimColor("#2f9e44"), ManimColor("#228be6"), ManimColor("#a5d8ff"), GREY]
lines_color = [YELLOW, ORANGE, PURPLE]

def traversal_scene(scene, LBTree, BTree, timeline):
    LBtags_seen = {}
    Linearized_tags = VGroup() 
    Arrows = VGroup()
    for i, (LBtag, line, Btag, tick) in enumerate(zip(LBTree.tags, LBTree.lines, BTree.Double_tags, timeline.ticks)):
        count = LBtags_seen.get(LBtag.number, 0) + 1
        LBtags_seen[LBtag.number] = count

        if isinstance(line, DashedLine):
            line = Line(line.get_start(), line.get_end())
        LBTreeDotAni = MoveAlongPath(scene.trace_dot_LBTree, line)
        BTreeCircleAni = scene.trace_circle_BTree.animate.move_to(Btag.get_center())

        LBtag_copy = LBtag.copy()
        LBtag_copy.order = count
        Linearized_tags.add(LBtag_copy)
        LBtag_copy.generate_target()
        LBtag_copy.target.move_to(tick.get_center())
        LBTreeTagAni = MoveToTarget(LBtag_copy)

        node_code = NodeCode(LBtag.copy())
        if count == 1:
            indication_color =  BLUE_C
            indicated_lines = node_code.to_be_displayed("preorder")
        elif count == 2:
            indication_color = YELLOW_D
            indicated_lines = node_code.to_be_displayed("inorder")
            if LBtag.number:
                for ln in node_code.to_be_displayed("preorder"):
                    ln.set_color(BLUE_C)
        elif count == 3:
            indication_color = RED_D
            indicated_lines = node_code.to_be_displayed("postorder")
            if LBtag.number:
                for ln in node_code.to_be_displayed("preorder"):
                    ln.set_color(BLUE_C)
                for ln in node_code.to_be_displayed("inorder"):
                    ln.set_color(YELLOW_D)
        if not LBtag.number:
            indication_color = BLUE_C
            indicated_lines = node_code.to_be_displayed("base")

        # FadeIn the code_block
        scene.play(FadeIn(node_code))
        scene.wait()
        indicate_steps_pre(scene, LBTree, color=YELLOW)

        arrow = Arrow(start=tick.get_bottom() + DOWN*1, end= tick.get_bottom() + DOWN*0.2, color = indication_color, stroke_width=12, buff=0, max_stroke_width_to_length_ratio=10)
        Arrows.add(arrow)
        arrow.order = count

        LBtag.z_index = max(scene.trace_dot_LBTree.z_index, LBtag_copy.z_index) + 1
        LBTree.entry_dot.z_index = scene.trace_dot_LBTree.z_index + 1
        # Move circle and dot, and then indicate tag, and then linearize tag, and then add arrow
        temp_copy = LBtag.outline.copy()
        temp_copy.z_index=LBtag.z_index+1
        temp_copy.set_fill(opacity=0)

        if i==0:
            scene.play(Create(scene.trace_circle_BTree))
        scene.play(Succession(AnimationGroup(BTreeCircleAni, LBTreeDotAni, lag_ratio=0, run_time=2), temp_copy.animate.set_stroke(color=YELLOW)))
        indicate_steps_unit(scene, LBTree, LBtag, i)
        scene.wait()
        scene.play(Succession(AnimationGroup(Indicate(LBtag, color=indication_color), Succession(Indicate(indicated_lines[0], color=indication_color), indicated_lines[0].animate.set_color(indication_color)))))
        scene.wait()
        scene.play(LBTreeTagAni)
        scene.wait()
        scene.play(GrowArrow(arrow))
        scene.wait(2)
        scene.play(Succession(Indicate(indicated_lines[1], color=indication_color), indicated_lines[1].animate.set_color(indication_color)))
        scene.wait()
        scene.play(FadeOut(node_code), FadeOut(temp_copy))
        scene.wait()
        if i==3:
            break
    scene.play(FadeOut(scene.trace_circle_BTree, scene.trace_dot_LBTree))
    return VGroup(Linearized_tags, Arrows)

class Test(Scene):
    def construct(self):
        self.add(NumberPlane())
        self.add(Rectangle(width=9, height=4, color= RED, fill_opacity=0).set_stroke(color=RED, width=4).move_to([-2.5, 1, 0]))
        self.add(Rectangle(width=5, height=3, color= GREEN, fill_opacity=0).set_stroke(color=GREEN, width=4).move_to([4.5, -2.5,0]))
        txt1 = Tex("The solution", font_size=100)
        txt1.shift(UP)
        txt2 = Tex("(explained)", font_size=70)
        txt2_HI = add_highlight_to_text(txt2, 1, 9, color=GREEN, opacity=0.8)
        txt2_GP = VGroup(txt2, txt2_HI)
        txt1_2_GP  = VGroup(txt1, txt2_GP).arrange(direction=DOWN)
        self.play(Write(txt1_2_GP), run_time=2)
        self.wait()
        self.play(FadeOut(txt1_2_GP))
        self.wait()

        LBTree = LinearizedBTree(root=buildTree(dots_color, 3), x_start=-10, x_distance=0.9, y_start=-2 ,y_distance=1.5, dots_color=dots_color, lines_color=lines_color)
        LBTree.build_structure_with_entry(LBTree.root)
        LBTree.move_to([-2.5, 1, 0])
        LBTree.scale_all(0.7)

        BTree = BinaryTree(root=buildTree(dots_color, 3), x_start=0, x_distance=3, y_start=3 ,y_distance=1, dots_color=dots_color, lines_color=lines_color)
        BTree.build_structure(BTree.root)
        BTree.add_double_tags()
        BTree.move_to([4.5, -2.5,0])
        BTree.scale_all(0.5)
        
        timeline = NumberLine(
            x_range=[0, 12, 1],
            length=7,
            color=WHITE,
            include_numbers=False,
            stroke_width=5,
            tick_size=0.1,
            numbers_with_elongated_ticks=[0,12]
        )
        align_center(timeline, LBTree, 'x')
        timeline.shift(DOWN*2)
        for tick in timeline.ticks:
            tick.set_color(YELLOW)
        self.trace_dot_LBTree = Dot(color=YELLOW, radius=0.1).align_to(LBTree,DL)
        self.trace_circle_BTree = Circle(color=YELLOW, radius=BTree.tags[0].outline.radius+0.07).move_to(BTree.entry_dot.get_center())
        
        self.play(Create(BTree), Create(LBTree), Create(timeline), run_time=5)
        self.wait(2)
        traversal_scene_data = traversal_scene(self, LBTree, BTree, timeline)




Manim Community v0.19.0

[05/26/25 15:25:38] INFO     Animation 0 : Partial movie file written in                   ]8;id=169569;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=370653;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\Asser\Documents\Coding\VS                                                   
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\partial_                         
                             movie_files\Test\1185818338_1152674903_1478189169.mp4'                                

[05/26/25 15:25:39] INFO     Animation 1 : Partial movie file written in                   ]8;id=186420;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=720563;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\Asser\Documents\Coding\VS                                                   
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\partial_                         
                             movie_files\Test\624642324_2688965532_674487042.mp4'                                  

                    INFO     Animation 2 : Partial movie file written in                   ]8;id=128403;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=552174;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\Asser\Documents\Coding\VS                                                   
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\partial_                         
                             movie_files\Test\624642324_2060880192_2500557432.mp4'                                 

                    INFO     Animation 3 : Partial movie file written in                   ]8;id=933587;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=486880;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\Asser\Documents\Coding\VS                                                   
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\partial_                         
                             movie_files\Test\624642324_2688965532_1127025859.mp4'                                 

...

[05/26/25 15:29:41] INFO     Animation 64 : Partial movie file written in                  ]8;id=973345;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=793745;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\Asser\Documents\Coding\VS                                                   
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\partial_                         
                             movie_files\Test\624642324_2526679843_529661002.mp4'                                  

                    INFO     Combining to Movie file.                                      ]8;id=35096;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=663495;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

[05/26/25 15:29:43] INFO                                                                   ]8;id=376241;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=726348;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at 'C:\Users\Asser\Documents\Coding\VS                                     
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\Test.mp4                         
                             '                                                                                     
                                                                                                                   

                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=748989;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=823559;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#856\856]8;;\
                             manim has removed the 65 oldest file(s). You can change this                          
                             behaviour by changing max_files_cached in config.                                     

                    INFO     Rendered Test                                                             ]8;id=480041;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=245673;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 65 animations                                                                  

In [2]:
%%manim -ql Test

class Test(Scene):
    def construct(self):
        self.add(NumberPlane())
        BTree = BinaryTree(root = buildTree())
        BTree.build_structure(BTree.root)
        BTree.add_double_tags()
        BTree.scale_all(0.5)
        for i, item in enumerate(BTree):
            spread_in_grid(item, y_shift=i)
        self.add(BTree)
        print(BTree)
        self.wait()

Manim Community v0.19.0

BinaryTree of 5 submobjects


[05/26/25 13:51:59] INFO     Animation 0 : Partial movie file written in                   ]8;id=359012;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=969289;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\Asser\Documents\Coding\VS                                                   
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\partial_                         
                             movie_files\Test\1185818338_2688965532_3774900388.mp4'                                

                    INFO     Combining to Movie file.                                      ]8;id=521767;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=203969;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=656424;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=30102;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at 'C:\Users\Asser\Documents\Coding\VS                                     
                             CODE\Manim\BinaryTree\media\videos\BinaryTree\480p15\Test.mp4                         
                             '                                                                                     
                                                                                                                   

                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=768586;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=749251;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene_file_writer.py#856\856]8;;\
                             manim has removed the 1 oldest file(s). You can change this                           
                             behaviour by changing max_files_cached in config.                                     

                    INFO     Rendered Test                                                             ]8;id=957981;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=172638;file://C:\Users\Asser\AppData\Local\Programs\Python\Python311\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 1 animations                                                                   